In [11]:
import os
import pandas as pd

from cube.api import Cube
from utils.text_data_transformation import transform_raw_data
from utils.dialog_manipulation import prepare_dialogs

In [12]:
# 0) Firstly, use data_transformation.py to get "subdialogs_{id}.csv" files


# or DIALOG_IDS = [-1] for all in dir
DIALOG_IDS = [-1]
# DIALOG_IDS = ["138918380", "470409323", "347963763", "850978724", "511986933"]
path_to_dialogs_frame = os.path.join("data", "new_type_dialogs_prepared2", "general_df.csv")
path_to_save_result = os.path.join("data", "processed_dialog_files", "subdialog_keywords.csv")

# flag_get_all = 0
# if DIALOG_IDS[0] == -1:
#     DIALOG_IDS = os.listdir(path_to_prepared_dialogs)
#     flag_get_all = 1

frames = []
df_idf = pd.DataFrame()
general_n_subdialogs = 0


data_for_df = []
data = pd.read_csv(path_to_dialogs_frame)

n_subdialog = 0
dialog_dict, dialog_text = {}, ''
for index, row in data.iterrows():
    if n_subdialog != row.subdialog_id:
        n_subdialog += 1
        general_n_subdialogs += 1
        if n_subdialog != 1:
            dialog_dict["body"] = dialog_text
            data_for_df.append(dialog_dict)

        dialog_dict = {}
        dialog_text = ''
        # TODO: "body" delete from tf_idf
        # TODO: "title" delete from tf_idf
        dialog_dict["dialog_id"] = general_n_subdialogs
        dialog_dict["title"] = ""

    if not pd.isnull(row.preprocessed_message):
        dialog_text += " " + row.preprocessed_message

df_idf = pd.DataFrame(data_for_df)
df_idf

,dialog_id,title,body
0,1,,
1,2,,"тут мав би бути жарт про те який ти гарячий, ..."
2,3,,😂😂
3,4,,
4,5,,"Ну там не шибко гаряче, я тобі скажу...."
...,...,...,...
4202,4203,,добре дякувати
4203,4204,,треба
4204,4205,,добрий ранка потрібний тест мікрофон заздале...
4205,4206,,невідомо якийсь сповіщення може зарахуватися...


In [13]:
# # 1) write your number of rows to analyse. max == 695,
# # but to clean and analyse 30 rows is spending 6 minutes
# path_to_articles = os.path.join('data', 'processed_dialog_files', 'ukr_articles')
# data = pd.read_csv(path_to_articles, sep = ';', header = None)
# df_idf = pd.DataFrame(data[:20])
#
# df_idf.columns = ["id", "title", "en_title", "body", "date", "source_url",
#                   "article_url"]
#
# print("Schema:\n\n",df_idf.dtypes)
# print("Number of questions,columns=",df_idf.shape)
# df_idf

In [14]:

df_idf.dropna(subset = ["title"], inplace=True)
df_idf.dropna(subset = ["body"], inplace=True)

df_idf['text'] = df_idf['title'] + df_idf['body']

# if you have untransformed_data so uncomment it !!!!!!!!!!!!!
# if lang == "ua":
#     cube = Cube(verbose=True)
#     cube.load("uk")
# df_idf['text'] = df_idf['text'].apply(lambda x: transform_raw_data(x, lang, "words_frequency", cube))

#show the first 'text'
df_idf['text'][2]

' 😂😂'

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words():
    """load stop words """

    try:
        with open(os.path.join(os.getcwd(), "..", "dicts", "ukrainian_stopwords.txt"), "r", encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    except FileNotFoundError:
        with open(os.path.join(os.getcwd(), "dicts", "ukrainian_stopwords.txt"), "r",
                  encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    return stop_words

#load a set of stop words
stopwords=get_stop_words()

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of words,
#ignore words that appear in 85% of documents,
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

d:\python\envs\venv_telegram_analysis\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_io', 'analysis', 'dialogs', 'dicts', 'encoding', 'mode', 'my_work', 'name', 'programming', 'telegram', 'textiowrapper', 'txt', 'ukrainian_stopwords', 'utf'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [16]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000, ngram_range=(2, 2))
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(4207, 10000)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# get test docs into a list
docs_test=df_idf['text'].tolist()
docs_title=df_idf['title'].tolist()
docs_body=df_idf['body'].tolist()

In [18]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====


=====Body=====


===Keywords===


In [19]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [21]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):

    # get vector for a single document
    curr_vector=tf_idf_vector[i]

    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)


    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
for result in results:
    result2 = {k: v for k, v in sorted(result.items(), key=lambda item: item[1], reverse=True)}
    # print(result2)

df.to_csv(path_to_save_result, index=False)
print("result df")
df

result df


,doc,keywords
0,,{}
1,"тут мав би бути жарт про те який ти гарячий, ...","{'який ти': 0.302, 'тут мав': 0.302, 'ти гаряч..."
2,😂😂,{}
3,,{}
4,"Ну там не шибко гаряче, я тобі скажу....","{'шибко гаряче': 0.408, 'тобі скажу': 0.408, '..."
...,...,...
4202,добре дякувати,{'добре дякувати': 1.0}
4203,треба,{}
4204,добрий ранка потрібний тест мікрофон заздале...,"{'тест мікрофон': 0.51, 'ранка потрібний': 0.5..."
4205,невідомо якийсь сповіщення може зарахуватися...,"{'якийсь сповіщення': 0.456, 'сповіщення може'..."
